In [4]:
import distanceCorrelation as dc
import numpy as np
import math
import pandas as pd
import random
import sklearn.gaussian_process as gp
import heapq
import copy
import scipy.stats as sp
import sys
import subprocess
import inspect
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import warnings


#https://www.geeksforgeeks.org/using-jupyter-notebook-in-virtual-environment/

In [5]:
df = pd.read_excel("input.xlsx")
df.drop(df[df.Group!="Ctrl"].index, inplace=True)
df.drop("Group", axis=1, inplace=True)
print(df)

    Glycerol  2-Hydroxybutyrate  2-Hydroxyisovalerate  3-Hydroxybutyrate  \
0    15669.4              205.2                  56.5              361.5   
1    10084.3              220.9                  92.6              438.0   
2    12836.6              394.7                  93.9              861.2   
3    10520.2              293.0                 200.1             1309.9   
4    15809.9              573.9                 131.3              580.0   
5    12581.0              182.0                  56.8              214.0   
6    12276.4             1103.3                 323.0              422.2   
7    13939.5              488.6                 156.6              493.3   
8    11628.6              567.3                  85.5              377.5   
9    17668.2             1043.2                 178.5              594.8   
10   10293.1              230.4                  52.6             1039.2   
11    6676.8              190.9                  62.1              224.7   
12    8192.4

In [14]:
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

kernel= 1.0*gp.kernels.RBF(1.0)

#models=[]
loss=[] #to be 2D array for all loss values
count=0.0

for molecule1 in df:
    count+=1
    print(str(round(count*100/len(df),2)) + "% complete")
    #molecule1models=[]
    molecule1Loss=[] #1D array, loss values for single molecule
    for molecule2 in df:
        if molecule1==molecule2: #ensure we don't compare a molecule to itself
            molecule1Loss+=[0.0]
            #molecule1models+=NULL

        else:
            #Splitting into train and validation sets
            x = df[molecule1].values
            x, cvx = np.split(x, [int(.8*len(x))])

            x = x.reshape(-1,1) #turn to 2D array
            cvx = cvx.reshape(-1,1)

            y = df[molecule2]
            y, cvy = np.split(y, [int(.8*len(y))])

            model = gp.GaussianProcessRegressor(kernel=kernel).fit(x,y)
            molecule1Loss+=[model.score(cvx,cvy)]

    loss+=[molecule1Loss]

3.33% complete
6.67% complete
10.0% complete
13.33% complete
16.67% complete
20.0% complete
23.33% complete
26.67% complete
30.0% complete
33.33% complete
36.67% complete
40.0% complete
43.33% complete
46.67% complete
50.0% complete
53.33% complete
56.67% complete
60.0% complete
63.33% complete
66.67% complete
70.0% complete
73.33% complete
76.67% complete
80.0% complete
83.33% complete
86.67% complete
90.0% complete
93.33% complete
96.67% complete
100.0% complete
103.33% complete
106.67% complete
110.0% complete
113.33% complete
116.67% complete
120.0% complete
123.33% complete
126.67% complete
130.0% complete
133.33% complete
136.67% complete
140.0% complete
143.33% complete
146.67% complete
150.0% complete
153.33% complete
156.67% complete
160.0% complete
163.33% complete
166.67% complete
170.0% complete
173.33% complete
176.67% complete
180.0% complete
183.33% complete
186.67% complete
190.0% complete
193.33% complete
196.67% complete
200.0% complete
203.33% complete
206.67% comple

In [26]:
count=0

for i in range(len(loss)):
    if loss[i]>0:
        print(df.columns[i//218]+" -> "+df.columns[i%218])
        count+=1


print(str(count) + " molecules have a score above 0.5")

3-Hydroxybutyrate -> C16:1
Acetoacetate -> C4
Acetoacetate -> C5:1-DC
Acetoacetate -> Trp
Acetoacetate -> Tyr
Acetoacetate -> Pro
Acetoacetate -> lysoPC a C18:1
Pyruvate -> C3-OH
UDP-galactose -> Propylene glycol
UDP-galactose -> Fructose
UDP-galactose -> C14:1
UDP-galactose -> C16
UDP-galactose -> Ile
UDP-galactose -> Leu
UDP-galactose -> Phe
UDP-galactose -> Tyr
UDP-galactose -> Histamine
UDP-galactose -> Ser
UDP-galactose -> Carnosine
UDP-galactose -> Serotonin
UDP-galactose -> lysoPC a C18:2
C3-OH -> O-Phosphocholine
C3-OH -> Urea
C3-OH -> Xanthine
C3-OH -> C0
C3-OH -> C2
C3-OH -> C18:1
C3-OH -> Met
C3:1 -> Acetoacetate
C3:1 -> Formate
C3:1 -> Inosine
C3:1 -> C6 (C4:1-DC)
C3:1 -> PC aa C42:1
C3:1 -> PC ae C40:4
C4:1 -> C3-OH
C5 -> Cit
C5-M-DC -> C2
C5-M-DC -> C3
C5-M-DC -> C4
C5-M-DC -> C7-DC
C5-M-DC -> C16-OH
C5-M-DC -> C16:2
C5-M-DC -> C18:1
C5-M-DC -> C18:2
C5-M-DC -> Met
C5-M-DC -> Carnosine
C5-OH (C3-DC-M) -> C16-OH
C5-OH (C3-DC-M) -> C16:2
C5-OH (C3-DC-M) -> Leu
C5:1 -> C3-OH